# DEPENDENCIES

In [1]:
import numpy as np
import pandas as pd
import pandas_gbq
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (20, 12)

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyoff
import plotly.io as pio
pio.renderers.default = 'iframe'

### Connect to BQ Kraken Database

In [2]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = '/Users/asani/Downloads/octopus-data-prod-294206-8028c6b220ce.json'

In [3]:
project_id = 'octopus-data-prod-294206'
sql = """
SELECT *
FROM `octopus-data-prod-294206.KRAKEN_temp_view.customer_orders`
"""
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [4]:
df.head()

,order_id,system,timestamp,total_quantity,unit,amount_per_qty,total_amount,total_weight,user_id,receiver_id,...,latitude,longitude,country,province,city,district,village,order_status,route_id,total_distance
0,c21c5978-ee5b-41a9-b986-44d4528af525,V2,2021-07-15 09:14:44.294846,8.000000000,None,7.142857143,26.000000000,1770.830000000,71427,<NA>,...,-7.000984,107.560314,Indonesia,Jawa Barat,Bandung,Kecamatan Katapang,Sangkanhurip,cancelled,<NA>,NaN
1,8cb3d879-4f8f-453f-9d83-d056426801c2,V2,2021-05-22 11:31:39.273823,10.000000000,None,25.000000000,250.000000000,2500.000000000,41305,<NA>,...,-6.930185,107.744194,Indonesia,Jawa Barat,Bandung,Cileunyi,Cimekar,cancelled,<NA>,NaN
2,a1922eef-bf00-41b1-bc72-ccaea8c62ec1,V2,2021-06-06 09:13:58.296980,50.000000000,None,29.142857143,350.000000000,8994.209000000,41305,<NA>,...,-6.930185,107.744194,Indonesia,Jawa Barat,Bandung,Cileunyi,Cimekar,cancelled,<NA>,NaN
3,9b076ad4-91d2-4d69-aa9c-2d3feb47c1a8,V2,2021-08-01 22:14:18.892684,50.000000000,None,1.000000000,50.000000000,192.500000000,62302,<NA>,...,-7.044907,107.540225,Indonesia,Jawa Barat,Bandung,Kecamatan Cangkuang,Ciluncat,cancelled,<NA>,NaN
4,fa60b03a-cfde-4eac-81da-a98aaa0b4f3c,V2,2021-06-22 12:13:55.076571,28.000000000,None,8.928571429,250.000000000,7000.000000000,67965,<NA>,...,-6.947300,107.736110,Indonesia,Jawa Barat,Bandung,Cileunyi,Cimekar,cancelled,<NA>,NaN


In [5]:
# Data info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49689 entries, 0 to 49688
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        49689 non-null  object        
 1   system          49689 non-null  object        
 2   timestamp       49689 non-null  datetime64[ns]
 3   total_quantity  48935 non-null  object        
 4   unit            4629 non-null   object        
 5   amount_per_qty  48935 non-null  object        
 6   total_amount    48935 non-null  object        
 7   total_weight    48935 non-null  object        
 8   user_id         49689 non-null  Int64         
 9   receiver_id     33698 non-null  Int64         
 10  region_id       49689 non-null  Int64         
 11  region_name     48331 non-null  object        
 12  latitude        49689 non-null  float64       
 13  longitude       49689 non-null  float64       
 14  country         48357 non-null  object        
 15  pr

In [6]:
# Check missing value

df.isnull().sum()

order_id              0
system                0
timestamp             0
total_quantity      754
unit              45060
amount_per_qty      754
total_amount        754
total_weight        754
user_id               0
receiver_id       15991
region_id             0
region_name        1358
latitude              0
longitude             0
country            1332
province           1358
city               1358
district           1430
village            1384
order_status          0
route_id          20363
total_distance    20363
dtype: int64

In [7]:
# Check Order Status

df.order_status.value_counts()

completed    37008
cancelled    12681
Name: order_status, dtype: int64

# DATA PREPROCESSING

In [8]:
df.loc[:,['total_quantity','total_amount', 'total_weight']].astype('float')

,total_quantity,total_amount,total_weight
0,8.0,26.0,1770.830
1,10.0,250.0,2500.000
2,50.0,350.0,8994.209
3,50.0,50.0,192.500
4,28.0,250.0,7000.000
...,...,...,...
49684,2.5,1000.0,2500.000
49685,1.0,15000.0,1000.000
49686,9.1,30220.0,9100.000
49687,0.5,375.0,500.000


In [9]:
# Datetime Preprocessing

def date_preprocessing(df):
    df = df.copy().loc[:,['order_id', 'user_id', 'timestamp', 'total_quantity', 'total_amount', 'total_weight', 
         'order_status', 'province', 'city']]
    #df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['day'] = df['timestamp'].dt.day
    df['day_name'] = df['timestamp'].dt.day_name()
    df['month'] = df['timestamp'].dt.month
    df['year'] = df['timestamp'].dt.year
    df['month_year'] = df['timestamp'].dt.to_period('M').astype(str)
    df['total_amount'] = df['total_amount'].astype('float')
    df['total_quantity'] = df['total_quantity'].astype('float')
    df['total_weight'] = df['total_weight'].astype('float')
    return df

In [10]:
# Convert timestamp to datetime

df = date_preprocessing(df)
df.head()

,order_id,user_id,timestamp,total_quantity,total_amount,total_weight,order_status,province,city,day,day_name,month,year,month_year
0,c21c5978-ee5b-41a9-b986-44d4528af525,71427,2021-07-15 09:14:44.294846,8.0,26.0,1770.830,cancelled,Jawa Barat,Bandung,15,Thursday,7,2021,2021-07
1,8cb3d879-4f8f-453f-9d83-d056426801c2,41305,2021-05-22 11:31:39.273823,10.0,250.0,2500.000,cancelled,Jawa Barat,Bandung,22,Saturday,5,2021,2021-05
2,a1922eef-bf00-41b1-bc72-ccaea8c62ec1,41305,2021-06-06 09:13:58.296980,50.0,350.0,8994.209,cancelled,Jawa Barat,Bandung,6,Sunday,6,2021,2021-06
3,9b076ad4-91d2-4d69-aa9c-2d3feb47c1a8,62302,2021-08-01 22:14:18.892684,50.0,50.0,192.500,cancelled,Jawa Barat,Bandung,1,Sunday,8,2021,2021-08
4,fa60b03a-cfde-4eac-81da-a98aaa0b4f3c,67965,2021-06-22 12:13:55.076571,28.0,250.0,7000.000,cancelled,Jawa Barat,Bandung,22,Tuesday,6,2021,2021-06


In [11]:
# Check Rows that total_quantity, total_amount, and total_weight are NA Based on order_status

order_missing = df[df[['total_quantity', 'total_amount','total_weight']].isna().any(axis=1)]
order_missing.order_status.value_counts()

cancelled    730
completed     24
Name: order_status, dtype: int64

In [12]:
# Check Rows that total_quantity, total_amount, and total_weight are NA Based on order_status

order_missing[order_missing['order_status']=='completed'].head()

,order_id,user_id,timestamp,total_quantity,total_amount,total_weight,order_status,province,city,day,day_name,month,year,month_year
6938,33784908-0b0d-4d30-91da-84b685bffc84,74219,2021-09-11 11:09:23.129378,NaN,NaN,NaN,completed,Jawa Barat,Kota Bandung,11,Saturday,9,2021,2021-09
8259,bb90deb0-ed18-4b1d-a667-0e933ad9e425,15744,2021-08-29 20:50:13.956088,NaN,NaN,NaN,completed,Sulawesi Selatan,Kabupaten Gowa,29,Sunday,8,2021,2021-08
10647,633dac7f-d76e-4b25-a2bf-69ac67e7fdc2,73886,2021-08-30 16:14:56.466087,NaN,NaN,NaN,completed,Jawa Barat,Bandung,30,Monday,8,2021,2021-08
14472,ded041c9-8fc6-4729-ad47-d23f0a69a77f,15744,2021-08-30 05:00:20.483905,NaN,NaN,NaN,completed,Bali,Kabupaten Badung,30,Monday,8,2021,2021-08
16080,509c5310-fd6f-4808-901a-09bb5e4bb253,75182,2021-10-01 15:37:42.956597,NaN,NaN,NaN,completed,Jawa Barat,Kota Bandung,1,Friday,10,2021,2021-10


* There were 24 completed orders which *total quantity*, *total_amount*, and *total_weight* are missing (N/A).
* We investigate it on psql backend prod database, the listed order_id is not captured on v2.order_item, so we assume there's bug in backend process.
* Hence, we'll drop these missing values for now. (Already confirmed by tech and product team)

In [13]:
# Remove missing values of quantity, amount, and weight

df = df.dropna(subset=['total_quantity', 'total_amount','total_weight'])
df.isnull().sum()

order_id             0
user_id              0
timestamp            0
total_quantity       0
total_amount         0
total_weight         0
order_status         0
province          1266
city              1266
day                  0
day_name             0
month                0
year                 0
month_year           0
dtype: int64

In [14]:
# Filter out cancelled order

df_completed = df[df['order_status']=='completed']
df_completed.head()

,order_id,user_id,timestamp,total_quantity,total_amount,total_weight,order_status,province,city,day,day_name,month,year,month_year
410,86d49f24-8d60-48d5-835e-2904d666380b,64836,2021-06-06 11:13:15.346324,10.0,250.0,2500.000,completed,Jawa Barat,Bandung,6,Sunday,6,2021,2021-06
411,3b9407b7-0620-4d21-9d4b-11b6472ac034,55179,2021-05-22 09:21:32.196413,7.0,250.0,1750.000,completed,Jawa Barat,Bandung,22,Saturday,5,2021,2021-05
412,6b0be3d2-a808-4a85-b812-6f85e8999a46,67497,2021-06-11 13:57:21.780340,4.0,24.0,54.040,completed,Jawa Barat,Bandung,11,Friday,6,2021,2021-06
413,a0a44ac6-ab55-492a-96b6-855922268da2,40978,2021-06-06 07:47:43.288231,51.0,270.0,12042.171,completed,Jawa Barat,Kabupaten Bandung Barat,6,Sunday,6,2021,2021-06
414,16ddd6b5-6fd6-456b-a383-a90e0dcde873,59693,2021-07-21 08:20:55.474495,3.0,28.0,513.510,completed,Jawa Barat,Bandung,21,Wednesday,7,2021,2021-07


In [15]:
df_completed.loc[:,['total_quantity', 'total_amount','total_weight']].describe()

,total_quantity,total_amount,total_weight
count,36984.000000,3.698400e+04,3.698400e+04
mean,39.238614,2.280502e+04,6.040190e+03
std,65.738503,1.862384e+05,2.326576e+04
min,0.000000,0.000000e+00,0.000000e+00
25%,8.500000,2.360000e+02,5.550000e+02
50%,21.000000,2.960000e+02,2.151954e+03
75%,48.000000,1.142000e+03,5.783368e+03
max,2450.000000,9.815420e+06,1.540000e+06


Here we can see the that:
1. Minimal total_quantity, total amount, and total weight are zero (make no sense).  Thus I'll filter the transaction that greater than 0.
2. Since the unit for total quantity is not the same, I'll skip this for now.
2. The median and mean for total amount and total weight are far apart. For instance, the median and average of total weight of customer orders are 2,500 gr and 6,149.6 gr respectively. This wide gap might occurs due to type of waste that user assign (some might assign plastics waste and some might assign electronic waste like pc)

In [16]:
# Distribution of total_amount and total_weight Before Filtering Out Zero Values

fig = go.Figure()
fig.add_trace(go.Box(y=df_completed['total_amount'], name='Total Amount (points)'))
fig.add_trace(go.Box(y=df_completed['total_weight'], name='Total Weight (grams)'))
fig.update_layout(title='Distribution of Total Amount and Total Weight')
pio.base_renderers.default = "browser"
fig.show()

In [17]:
# Distribution of total_amount and total_weight After Filtering Out Zero Values

fig = go.Figure()
fig.add_trace(go.Box(y=df_completed['total_amount'][df_completed['total_amount']>0], name='Total Amount (points)', boxmean=True))
fig.add_trace(go.Box(y=df_completed['total_weight'][df_completed['total_weight']>0], name='Total Weight (grams)', boxmean=True))
fig.update_layout(title='Distribution of Total Amount and Total Weight')
fig.show()

In [18]:
# Distribution of Amount and Weight After Transforming to Log Scale

fig = go.Figure()
fig.add_trace(go.Box(y=np.log(df_completed['total_amount'][df_completed['total_amount']>0]), name='Total Amount (points)', boxmean=True))
fig.add_trace(go.Box(y=np.log(df_completed['total_weight'][df_completed['total_weight']>0]), name='Total Weight (grams)', boxmean=True))
fig.update_layout(title='Distribution of Total Amount and Total Weight (log)')
fig.show()

In [19]:
df_completed.loc[:,['total_amount','total_weight']][(df_completed.total_amount > 0) & (df_completed.total_weight > 0)].describe()

,total_amount,total_weight
count,3.639000e+04,3.639000e+04
mean,2.317727e+04,5.650292e+03
std,1.877293e+05,2.035456e+04
min,1.000000e+00,2.000000e-02
25%,2.500000e+02,5.404000e+02
50%,3.000000e+02,2.090750e+03
75%,1.200000e+03,5.743060e+03
max,9.815420e+06,1.540000e+06


In [20]:
len(df_completed[df_completed['total_amount']==0]), len(df_completed[df_completed['total_weight']==0])

(593, 26)

What can we get from boxplot above:
1. There are many extreme values (outliers) for total amounts, and total weight. The cause might be due to type of waste customer assigned. 
2. After performing log transformation total weight is normal-enough distributed.
3. Total amount and weight is right skewed distributed. It means that total points gained by user orders is mostly below the average point of 7,727.95

In [21]:
df_completed = df_completed[(df_completed['total_amount']>0) & (df_completed['total_weight']>0)]

In [22]:
%store df_completed

Stored 'df_completed' (DataFrame)


In [23]:
%store df

Stored 'df' (DataFrame)
